In [1]:
import numpy as np
from scipy.spatial import distance
import time

In [2]:
start_time = time.time()

In [3]:
N = 4 #four states in eache one
P1 = np.array([[0.3,0.4,0.2,0.1], [0.2,0.3,0.5,0.0], [0.1,0.0,0.8,0.1], [0.4,0.0,0.0,0.6]]) #transition probability matrix
P2 = np.array([[0.1,0.1,0.1,0.7], [0.1,0.1,0.1,0.7], [0.1,0.1,0.1,0.7], [0.,0.,0.,1.0]])

In [4]:
#actions : choose bandit 1 or 2
actions = [1,2]

In [5]:
def R1(x):
    return x+7

def R2(x):
    if x == 3:
        return 0.
    else:
        return x+17

In [6]:
def RewardFunc(s,a): #reward function
    if a == 1:
        return R1(s[0])
    else:
        return R2(s[1])

In [7]:
def ProbTrans(r,s,a): #probability transition function
    if a == 1 and r[1] == s[1]:
        return P1[s[0],r[0]]
    elif a == 2 and r[0] == s[0]:
        return P2[s[1],r[1]]
    else:
        return 0.

In [8]:
def ExpectedVal(s,a,u): #expected value
    suma = 0.
    for i in range(0,N):
        for j in range(0,N):
            suma += ProbTrans((i,j),s,a)*u[(i,j)]
    return suma

In [9]:
def BellmanOp(x, u, actions_set = actions): #Bellman operator
    values = {"max" : -np.inf, "argmax" : []}
    for a in actions_set:
        B = RewardFunc(x,a) + g*ExpectedVal(x,a,u)
        
        if values["max"] == B:
            values["argmax"].append(a)
            
        elif values["max"] < B:
            values["max"] = B
            values["argmax"] = [a]
            
    return values

In [10]:
vn = np.zeros((N,N)) #start v0 as an array of zeroes
vni = np.zeros((N,N))
decision_grid = []

iterations = 1

e = 0.15 #epsilon
g = 0.95 #gamma

In [11]:
#first iteration
for i in range(0,N):
    for r in range(0,N): #state x = (i,j)
        values = BellmanOp((i,r), vn)
        vni[i,r] = values["max"]
        decision_grid.append({"index" : (i,r), "argmax" : values["argmax"]})

In [12]:
while distance.chebyshev(np.ravel(vni), np.ravel(vn)) >= e*(1-g)/(2*g):
    vn = np.copy(vni)
    decision_grid = []
    for i in range(0,N):
        for r in range(0,N): #state x = (i,j)
            values = BellmanOp((i,r), vn)
            vni[i,r] = values["max"]
            decision_grid.append({"index" : (i,r), "argmax" : values["argmax"]})
            
    iterations += 1

In [13]:
print(time.time() - start_time) #time needed for running the notebook

0.13083648681640625


In [14]:
print("Number of iterations: ", iterations)

Number of iterations:  151


In [15]:
#save decision rule
import json

with open('2-bandit_drule.txt', 'w') as f:
    f.write(json.dumps(decision_grid))

In [16]:
decision_grid

[{'index': (0, 0), 'argmax': [2]},
 {'index': (0, 1), 'argmax': [2]},
 {'index': (0, 2), 'argmax': [2]},
 {'index': (0, 3), 'argmax': [1]},
 {'index': (1, 0), 'argmax': [2]},
 {'index': (1, 1), 'argmax': [2]},
 {'index': (1, 2), 'argmax': [2]},
 {'index': (1, 3), 'argmax': [1]},
 {'index': (2, 0), 'argmax': [2]},
 {'index': (2, 1), 'argmax': [2]},
 {'index': (2, 2), 'argmax': [2]},
 {'index': (2, 3), 'argmax': [1]},
 {'index': (3, 0), 'argmax': [2]},
 {'index': (3, 1), 'argmax': [2]},
 {'index': (3, 2), 'argmax': [2]},
 {'index': (3, 3), 'argmax': [1]}]

In [17]:
print(np.around(vni,2))

[[182.67 183.67 184.67 170.42]
 [184.05 185.05 186.05 171.9 ]
 [185.77 186.77 187.77 173.76]
 [185.86 186.86 187.86 173.85]]
